In [1]:
import tensorflow as tf
import numpy as np

In [2]:
%run cnn_utils.py

In [3]:
batch_size = 100
shuffle_buffer =10000
num_epochs = 4
test_batch_szie = 15000 # 全部用于测试
test_epoch = 1

In [4]:
params = batch_size, shuffle_buffer, num_epochs
iterator_train = get_iterator(params, '../data/train_sets.tfrecords')
train_image_batch, train_label_batch = iterator_train.get_next()

params = batch_size, shuffle_buffer, test_epoch
iterator_test = get_iterator(params, '../data/test_sets.tfrecords')
test_image_batch, test_label_batch = iterator_test.get_next()

X = tf.placeholder(tf.float32, shape=[None, 48, 120, 1], name='x-input')
y_ = tf.placeholder(tf.int64, shape=[None, 208], name='y-input')
y_ = tf.cast(y_, tf.float32)
keep_prob = tf.placeholder(tf.float32, name='keep_prob')
y_pred = inference(X, keep_prob)
cross_entropy = -tf.reduce_sum(y_ * tf.log(tf.clip_by_value(y_pred, 1e-10, 1.0)))
losses = cross_entropy + tf.add_n(tf.get_collection('losses'))
# global_step = tf.Variable(0, trainable=False)
# learning_rate = tf.train.exponential_decay(
# 0.001,
# global_step,
# 5000, # 所有数据训练完所需的轮数
# 0.99
# )
train_step = tf.train.AdamOptimizer(0.001).minimize(losses)
accuracy = calc_accuracy(y_pred, y_)

sess = tf.InteractiveSession()
sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
sess.run(iterator_train.initializer)
train_count = 0
while True:
    train_count += 1
    try:
        X_train, y_train = sess.run([train_image_batch, train_label_batch])
        if train_count % 100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={X:X_train, y_: y_train, keep_prob: 1})
            print('train steps: %d, accuracy: %.3f' % (train_count, train_accuracy))
        sess.run(train_step, feed_dict={X:X_train, y_: y_train, keep_prob: 0.5})
    except Exception as e:
        break
# run test sets
sess.run(iterator_test.initializer)
test_accuracies = []
while True:
    try:
        X_test, y_test = sess.run([test_image_batch, test_label_batch])
        test_accuracy = sess.run(accuracy, feed_dict={X:X_test, y_: y_test, keep_prob: 1})
        test_accuracies.append(test_accuracy)
    except Exception as e:
        break
print('test acuracy: %.3f' % np.mean(test_accuracies))

train steps: 100, accuracy: 0.000
train steps: 200, accuracy: 0.000
train steps: 300, accuracy: 0.000
train steps: 400, accuracy: 0.010
train steps: 500, accuracy: 0.020
train steps: 600, accuracy: 0.010
train steps: 700, accuracy: 0.120
train steps: 800, accuracy: 0.110
train steps: 900, accuracy: 0.250
train steps: 1000, accuracy: 0.250
train steps: 1100, accuracy: 0.270
train steps: 1200, accuracy: 0.320
train steps: 1300, accuracy: 0.360
train steps: 1400, accuracy: 0.360
train steps: 1500, accuracy: 0.350
train steps: 1600, accuracy: 0.470
train steps: 1700, accuracy: 0.460
train steps: 1800, accuracy: 0.500
train steps: 1900, accuracy: 0.450
train steps: 2000, accuracy: 0.510
train steps: 2100, accuracy: 0.460
train steps: 2200, accuracy: 0.530
train steps: 2300, accuracy: 0.540
train steps: 2400, accuracy: 0.600
train steps: 2500, accuracy: 0.580
train steps: 2600, accuracy: 0.600
train steps: 2700, accuracy: 0.590
train steps: 2800, accuracy: 0.530
train steps: 2900, accuracy: 

In [5]:
X

<tf.Tensor 'x-input:0' shape=(?, 48, 120, 1) dtype=float32>

In [6]:
y_

<tf.Tensor 'Cast:0' shape=(?, 208) dtype=float32>

In [7]:
keep_prob

<tf.Tensor 'keep_prob:0' shape=<unknown> dtype=float32>

In [8]:
y_pred

<tf.Tensor 'y_pred:0' shape=(?, 208) dtype=float32>